<a href="https://colab.research.google.com/github/munich-ml/MLPy2020/blob/master/35_batch_evaluate_fMNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Intro

## References
Resources used to create this notebook:
- [scikit-learn website](https://scikit-learn.org)
- [Matplotlib website](https://matplotlib.org/)
- [Wikipedia](https://en.wikipedia.org/wiki/Main_Page)
- Hands-on Machine Learning with Scikit-learn, Keras & TensorFlow, Aurelien Geron, [Book on Amazon](https://www.amazon.de/Aur%C3%A9lien-G%C3%A9ron/dp/1492032646/ref=sr_1_3?__mk_de_DE=%C3%85M%C3%85%C5%BD%C3%95%C3%91&dchild=1&keywords=Hands-on+Machine+Learning+with+Scikit-learn%2C+Keras+%26+TensorFlow%2C+Aurelien+Geron%2C&qid=1589875241&sr=8-3)
- Introduction to Machine Learning with Python, Andreas Mueller, [Book on Amazon](https://www.amazon.de/Introduction-Machine-Learning-Python-Scientists/dp/1449369413)


## Setup

First, do the common imports.

Tensorflow must be 2.x, because there are major changes from 1.x

In [0]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Forces tensorflow version (only in colab)
%tensorflow_version 2.x           

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

# Common imports
import os
import pickle
import pandas as pd

# Setup matplotlib
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

# Get munich-ml helper functions from GitHub
if "MLPy2020" in os.listdir():
    !git -C MLPy2020 pull
else:
    !git clone https://github.com/munich-ml/MLPy2020/

from MLPy2020.MLPy_helper_funcs import pickle_in

Cloning into 'MLPy2020'...
remote: Enumerating objects: 41, done.
remote: Counting objects: 100% (41/41), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 259 (delta 21), reused 4 (delta 2), pack-reused 218
Receiving objects: 100% (259/259), 3.02 MiB | 5.66 MiB/s, done.
Resolving deltas: 100% (137/137), done.


## Mount google drive

In [0]:

mount_dir = os.path.join(os.getcwd(), "drive")
mount_dir

'/content/drive'

In [0]:
from google.colab import drive
drive.mount(mount_dir)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


#Batch Evaluation

In [0]:
save_dir = os.path.join(mount_dir, "My Drive", "Colab Notebooks", "models")
save_dir

'/content/drive/My Drive/Colab Notebooks/models'

In [0]:
from time import time
models = {}
for fn in os.listdir(save_dir):
    if fn.endswith(".h5"):
        model = keras.models.load_model(os.path.join(save_dir, fn))
        models[fn] = {"params": model.count_params(), "layers": len(model.layers)}
        # load the testset
        testset = fn.split(".")[0] + "_data.pkl"
        if testset in os.listdir(save_dir):
            pickle_in(os.path.join(save_dir, testset), locals())
            print("{}, X_test.shape: {}".format(fn, X_test.shape))
        
            t0 = time()
            models[fn]["accuracy"] = model.evaluate(X_test, y_test)[1]
            models[fn]["tEval [s]"] = time() - t0
        print("")
pd.DataFrame(models).transpose()

pickle_in: Updating scope with the following variables:
['X_test', 'y_test', 'X_valid', 'y_valid', 'class_names']
fMNIST_CNN_v3_ageron.h5, X_test.shape: (10000, 28, 28, 1)
313/313 [==============================] - 1s 3ms/step - loss: 0.3067 - accuracy: 0.9007

pickle_in: Updating scope with the following variables:
['X_test', 'y_test', 'class_names', 'X_valid', 'y_valid']
fMNIST_NN_v2_ageron.h5, X_test.shape: (10000, 28, 28)
313/313 [==============================] - 1s 2ms/step - loss: 0.3360 - accuracy: 0.8841

pickle_in: Updating scope with the following variables:
['X_test', 'y_test', 'X_valid', 'y_valid', 'class_names']
fMNIST_CNN_v5_cmasch.h5, X_test.shape: (10000, 28, 28, 1)
313/313 [==============================] - 1s 2ms/step - loss: 0.2245 - accuracy: 0.9237

pickle_in: Updating scope with the following variables:
['X_test', 'y_test', 'X_valid', 'y_valid', 'class_names']
fMNIST_CNN_v6_cmasch_mod.h5, X_test.shape: (10000, 28, 28, 1)
313/313 [==============================] -

,params,layers,accuracy,tEval [s]
fMNIST_CNN_v3_ageron.h5,1413834.0,14.0,0.9007,1.052071
fMNIST_NN_v2_ageron.h5,266610.0,4.0,0.8841,0.678097
fMNIST_CNN_v5_cmasch.h5,939150.0,13.0,0.9237,0.918721
fMNIST_CNN_v6_cmasch_mod.h5,953550.0,13.0,0.9224,0.952670
